# सत्र 5 – मल्टी-एजेंट ऑर्केस्ट्रेटर

Foundry Local का उपयोग करके एक सरल दो-एजेंट पाइपलाइन (शोधकर्ता -> संपादक) का प्रदर्शन करता है।


### व्याख्या: निर्भरता स्थापना
`foundry-local-sdk` और `openai` स्थापित करता है, जो स्थानीय मॉडल एक्सेस और चैट पूर्णताओं के लिए आवश्यक हैं। यह प्रक्रिया इडेमपोटेंट है।


# परिदृश्य
एक न्यूनतम दो‑एजेंट ऑर्केस्ट्रेटर पैटर्न को लागू करता है:
- **शोधकर्ता एजेंट** संक्षिप्त तथ्यात्मक बुलेट्स एकत्र करता है
- **संपादक एजेंट** कार्यकारी स्पष्टता के लिए पुनर्लेखन करता है

साझा मेमोरी प्रति एजेंट, मध्यवर्ती आउटपुट का क्रमिक पासिंग, और एक सरल पाइपलाइन फ़ंक्शन का प्रदर्शन करता है। अधिक भूमिकाओं (जैसे, आलोचक, सत्यापनकर्ता) या समानांतर शाखाओं तक विस्तारित किया जा सकता है।

**पर्यावरण चर:**
- `FOUNDRY_LOCAL_ALIAS` - उपयोग करने के लिए डिफ़ॉल्ट मॉडल (डिफ़ॉल्ट: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - प्राथमिक एजेंट मॉडल (ALIAS को ओवरराइड करता है)
- `AGENT_MODEL_EDITOR` - संपादक एजेंट मॉडल (डिफ़ॉल्ट रूप से प्राथमिक)

**SDK संदर्भ:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**यह कैसे काम करता है:**
1. **FoundryLocalManager** स्वचालित रूप से Foundry Local सेवा शुरू करता है
2. निर्दिष्ट मॉडल डाउनलोड और लोड करता है (या कैश्ड संस्करण का उपयोग करता है)
3. इंटरैक्शन के लिए OpenAI-संगत एंडपॉइंट प्रदान करता है
4. प्रत्येक एजेंट विशेष कार्यों के लिए अलग मॉडल का उपयोग कर सकता है
5. अंतर्निहित पुनः प्रयास तर्क अस्थायी विफलताओं को सहजता से संभालता है

**मुख्य विशेषताएं:**
- ✅ स्वचालित सेवा खोज और प्रारंभिककरण
- ✅ मॉडल जीवनचक्र प्रबंधन (डाउनलोड, कैश, लोड)
- ✅ OpenAI SDK संगतता परिचित API के लिए
- ✅ एजेंट विशेषज्ञता के लिए मल्टी-मॉडल समर्थन
- ✅ पुनः प्रयास तर्क के साथ मजबूत त्रुटि हैंडलिंग
- ✅ स्थानीय अनुमान (कोई क्लाउड API आवश्यक नहीं)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### व्याख्या: कोर आयात और टाइपिंग
एजेंट संदेश संग्रहण के लिए डेटाक्लासेस और स्पष्टता के लिए टाइपिंग संकेतों को प्रस्तुत करता है। बाद के एजेंट क्रियाओं के लिए Foundry Local मैनेजर और OpenAI क्लाइंट को आयात करता है।


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### व्याख्या: मॉडल प्रारंभिकरण (SDK पैटर्न)
Foundry Local Python SDK का उपयोग मजबूत मॉडल प्रबंधन के लिए किया जाता है:
- **FoundryLocalManager(alias)** - सेवा को स्वचालित रूप से शुरू करता है और उपनाम द्वारा मॉडल लोड करता है
- **get_model_info(alias)** - उपनाम को ठोस मॉडल ID में परिवर्तित करता है
- **manager.endpoint** - OpenAI क्लाइंट के लिए सेवा का एंडपॉइंट प्रदान करता है
- **manager.api_key** - API कुंजी प्रदान करता है (स्थानीय उपयोग के लिए वैकल्पिक)
- विभिन्न एजेंटों (प्राइमरी बनाम एडिटर) के लिए अलग-अलग मॉडल का समर्थन करता है
- लचीलापन के लिए एक्सपोनेंशियल बैकऑफ के साथ अंतर्निहित पुनः प्रयास तर्क
- यह सुनिश्चित करने के लिए कनेक्शन सत्यापन कि सेवा तैयार है

**मुख्य SDK पैटर्न:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**लाइफसाइकल प्रबंधन:**
- प्रबंधकों को उचित सफाई के लिए वैश्विक रूप से संग्रहीत किया जाता है
- प्रत्येक एजेंट विशेषज्ञता के लिए एक अलग मॉडल का उपयोग कर सकता है
- स्वचालित सेवा खोज और कनेक्शन प्रबंधन
- विफलताओं पर एक्सपोनेंशियल बैकऑफ के साथ सहज पुनः प्रयास

यह सुनिश्चित करता है कि एजेंट ऑर्केस्ट्रेशन शुरू होने से पहले उचित प्रारंभिकरण हो।

**संदर्भ:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### व्याख्या: एजेंट और मेमोरी क्लासेस
हल्के `AgentMsg` को मेमोरी एंट्रीज़ के लिए और `Agent` को निम्नलिखित के लिए परिभाषित करता है:
- **सिस्टम भूमिका** - एजेंट का व्यक्तित्व और निर्देश
- **संदेश इतिहास** - बातचीत के संदर्भ को बनाए रखता है
- **act() मेथड** - उचित त्रुटि प्रबंधन के साथ क्रियाओं को निष्पादित करता है

एजेंट विभिन्न मॉडलों (प्राइमरी बनाम एडिटर) का उपयोग कर सकता है और प्रत्येक एजेंट के लिए अलग संदर्भ बनाए रखता है। यह पैटर्न सक्षम बनाता है:
- क्रियाओं के बीच मेमोरी को बनाए रखना
- प्रत्येक एजेंट के लिए लचीला मॉडल असाइनमेंट
- त्रुटि अलगाव और पुनर्प्राप्ति
- आसान चेनिंग और ऑर्केस्ट्रेशन


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### व्याख्या: समन्वित पाइपलाइन
दो विशेष एजेंट बनाता है:
- **शोधकर्ता**: प्राथमिक मॉडल का उपयोग करता है, तथ्यात्मक जानकारी एकत्र करता है
- **संपादक**: अलग मॉडल का उपयोग कर सकता है (यदि कॉन्फ़िगर किया गया हो), सामग्री को परिष्कृत और पुनर्लिखित करता है

`pipeline` फ़ंक्शन:
1. शोधकर्ता कच्ची जानकारी एकत्र करता है
2. संपादक इसे कार्यकारी-तैयार आउटपुट में परिष्कृत करता है
3. मध्यवर्ती और अंतिम परिणाम दोनों लौटाता है

यह पैटर्न सक्षम करता है:
- मॉडल विशेषज्ञता (अलग-अलग भूमिकाओं के लिए अलग-अलग मॉडल)
- बहु-चरणीय प्रसंस्करण के माध्यम से गुणवत्ता सुधार
- जानकारी परिवर्तन की ट्रेसबिलिटी
- अधिक एजेंटों या समानांतर प्रसंस्करण के लिए आसान विस्तार


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### व्याख्या: पाइपलाइन निष्पादन और परिणाम
एक अनुपालन + विलंबता विषयक प्रश्न पर मल्टी-एजेंट पाइपलाइन को निष्पादित करता है ताकि निम्नलिखित को प्रदर्शित किया जा सके:
- बहु-चरणीय जानकारी परिवर्तन
- एजेंट विशेषज्ञता और सहयोग
- परिष्करण के माध्यम से आउटपुट गुणवत्ता में सुधार
- ट्रेसबिलिटी (मध्यवर्ती और अंतिम आउटपुट दोनों संरक्षित)

**परिणाम संरचना:**
- `question` - मूल उपयोगकर्ता प्रश्न
- `research` - कच्चा शोध आउटपुट (तथ्यात्मक बिंदु)
- `final` - परिष्कृत कार्यकारी सारांश
- `models` - प्रत्येक चरण के लिए उपयोग किए गए मॉडल

**विस्तार के विचार:**
1. गुणवत्ता समीक्षा के लिए एक आलोचक एजेंट जोड़ें
2. विभिन्न पहलुओं के लिए समानांतर शोध एजेंट लागू करें
3. तथ्य-जांच के लिए एक सत्यापनकर्ता एजेंट जोड़ें
4. विभिन्न जटिलता स्तरों के लिए अलग-अलग मॉडल का उपयोग करें
5. पुनरावृत्त सुधार के लिए फीडबैक लूप लागू करें


### उन्नत: कस्टम एजेंट कॉन्फ़िगरेशन

एजेंट के व्यवहार को अनुकूलित करने के लिए प्रारंभिक सेल चलाने से पहले पर्यावरण चर को संशोधित करने का प्रयास करें:

**उपलब्ध मॉडल:**
- टर्मिनल में `foundry model ls` का उपयोग करके सभी उपलब्ध मॉडलों को देखें
- उदाहरण: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, आदि।


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
